In [ ]:
import numpy as np
import pandas as pd
import random
from sklearn.utils import shuffle

# === Παραμετρικές ρυθμίσεις ===
num_windows = 144          # Σύνολο παραθύρων
window_size = 5000         # Δείγματα ανά παράθυρο
sampling_interval = 0.01   # Δευτερόλεπτα ανά δείγμα
ambiguous_ratio = 0.3      # Ποσοστό “γκρίζων” παραθύρων (0.0–0.5)
difficulty_level = "easy"  # Επιλογές: "easy", "medium", "hard"

In [ ]:
# === Ρύθμιση θορύβου με βάση τη δυσκολία ===
def get_noise_levels(level):
    if level == "easy":
        return {"safe": 0.1, "risky": 0.6, "ambiguous": 0.3}
    elif level == "medium":
        return {"safe": 0.2, "risky": 0.45, "ambiguous": 0.35}
    elif level == "hard":
        return {"safe": 0.3, "risky": 0.35, "ambiguous": 0.35}
    else:
        return {"safe": 0.2, "risky": 0.45, "ambiguous": 0.35}

noise = get_noise_levels(difficulty_level)
total_samples = num_windows * window_size
time = np.round(np.arange(0, total_samples * sampling_interval, sampling_interval), 2)

In [ ]:
# === Συναρτήσεις προσομοίωσης παραθύρων ===
def simulate_window(mode):
    t = np.arange(0, window_size * sampling_interval, sampling_interval)

    if mode == "safe":
        x = np.random.normal(0, noise["safe"], window_size)
        y = np.random.normal(0.3, noise["safe"], window_size)
        z = np.random.normal(9.81, noise["safe"] * 0.5, window_size)

    elif mode == "risky":
        x = np.sin(t * 4) * 0.7 + np.random.normal(0, noise["risky"], window_size)
        y = np.sign(np.sin(t * 3)) * 1.2 + np.random.normal(0, noise["risky"], window_size)
        z = np.random.normal(10.2, noise["risky"], window_size)

    elif mode == "ambiguous":
        safe = simulate_window("safe")
        risky = simulate_window("risky")
        alpha = np.random.uniform(0.4, 0.6)
        return alpha * safe + (1 - alpha) * risky

    else:
        x = y = z = np.zeros(window_size)

    return np.stack([x, y, z], axis=1)

In [ ]:
# === Δημιουργία δεδομένων ===
num_ambiguous = int(num_windows * ambiguous_ratio)
num_clear = num_windows - num_ambiguous
num_safe = int(num_clear * 0.7)
num_risky = num_clear - num_safe

windows = []
labels = []

for _ in range(num_safe):
    windows.append(simulate_window("safe"))
    labels.append(0)

for _ in range(num_risky):
    windows.append(simulate_window("risky"))
    labels.append(1)

for _ in range(num_ambiguous):
    windows.append(simulate_window("ambiguous"))
    labels.append(random.choice([0, 1]))

# === Ανακάτεμα ===
combined = list(zip(windows, labels))
windows, labels = zip(*shuffle(combined, random_state=42))
data_array = np.vstack(windows)

# === Κατασκευή DataFrame ===
df = pd.DataFrame(data_array, columns=["Acceleration_X_Lateral", "Acceleration_Y_Longitudinal", "Acceleration_Z_Vertical"])
df.insert(0, "Global_Time_Seconds", time)

# === Εξαγωγή αρχείων ===
df.to_csv("acceleration_data.csv", index=False)
pd.DataFrame(labels).to_csv("empirical_labels.csv", header=False, index=False)

In [ ]:
print(f"Safe windows: {num_safe}")
print(f"Risky windows: {num_risky}")
print(f"Ambiguous windows: {num_ambiguous}")
print(f"Σύνολο παραθύρων: {num_safe + num_risky + num_ambiguous}")
print("Ολοκληρώθηκε με επιτυχία:")
print("- acceleration_data.csv")
print("- empirical_labels.csv")

Safe windows: 70
Risky windows: 31
Ambiguous windows: 43
Σύνολο παραθύρων: 144
Ολοκληρώθηκε με επιτυχία:
- acceleration_data.csv
- empirical_labels.csv


In [ ]:
import matplotlib.pyplot as plt

# Οπτικοποίηση ενός safe και ενός risky παραθύρου
example_safe = simulate_window("safe")
example_risky = simulate_window("risky")
t_window = np.linspace(0, 10, window_size)

fig, axs = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

axs[0].plot(t_window, example_safe[:, 0], label='X', color='blue')
axs[0].plot(t_window, example_safe[:, 1], label='Y', color='green')
axs[0].plot(t_window, example_safe[:, 2], label='Z', color='orange')
axs[0].set_title("Safe Window")
axs[0].legend()
axs[0].grid(True)

axs[1].plot(t_window, example_risky[:, 0], label='X', color='blue')
axs[1].plot(t_window, example_risky[:, 1], label='Y', color='green')
axs[1].plot(t_window, example_risky[:, 2], label='Z', color='orange')
axs[1].set_title("Risky Window")
axs[1].legend()
axs[1].grid(True)

plt.xlabel("Χρόνος (δευτ.)")
plt.tight_layout()
plt.savefig("sample_plot.png", dpi=300, bbox_inches="tight")
plt.close()
print("Το γράφημα αποθηκεύτηκε ως sample_plot.png")

Το γράφημα αποθηκεύτηκε ως sample_plot.png
